# Setup Dependencies and Class
Import required libraries (yfinance, pandas, plotly) and create a StockPlotter class.

In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

class StockPlotter:
    def __init__(self):
        pass

    def get_data(self, ticker, start_date, end_date):
        # Retrieve stock data from Yahoo Finance
        data = yf.download(ticker, start=start_date, end=end_date)
        return data

    def plot_data(self, data):
        # Create a plotly figure
        fig = go.Figure()

        # Add candlestick chart
        fig.add_trace(go.Candlestick(x=data.index,
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'],
                                     name='Candlestick'))

        # Update layout
        fig.update_layout(title='Stock Price Data',
                          xaxis_title='Date',
                          yaxis_title='Price',
                          xaxis_rangeslider_visible=False)

        fig.show()

# Fetch Stock Data
Use yfinance to download historical stock data including OHLCV (Open, High, Low, Close, Volume) information.

In [2]:
# Initialize the StockPlotter class
stock_plotter = StockPlotter()

# Define the stock ticker, start date, and end date
ticker = 'AAPL'
start_date = '2022-01-01'
end_date = '2023-01-01'

# Fetch the stock data
stock_data = stock_plotter.get_data(ticker, start_date, end_date)

# Display the first few rows of the data
stock_data.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2022-01-03 00:00:00+00:00,179.076599,182.009995,182.880005,177.710007,177.830002,104487900
2022-01-04 00:00:00+00:00,176.803787,179.699997,182.940002,179.119995,182.630005,99310400
2022-01-05 00:00:00+00:00,172.100876,174.919998,180.169998,174.639999,179.610001,94537600
2022-01-06 00:00:00+00:00,169.227936,172.000000,175.300003,171.639999,172.699997,96904000
2022-01-07 00:00:00+00:00,169.395187,172.169998,174.139999,171.029999,172.889999,86709100


# Calculate Technical Indicators
Implement technical indicators calculation including EMA, SMA, RSI, MACD, and Parabolic SAR using pandas operations.

In [3]:
def calculate_technical_indicators(data):
    # Calculate EMA
    data['EMA_8'] = data['Close'].ewm(span=8, adjust=False).mean()
    data['EMA_21'] = data['Close'].ewm(span=21, adjust=False).mean()

    # Calculate SMA
    data['SMA_10'] = data['Close'].rolling(window=10).mean()

    # Calculate RSI
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['RSI_14'] = 100 - (100 / (1 + rs))

    # Calculate MACD
    ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = ema_12 - ema_26
    data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate Parabolic SAR
    data['SAR'] = data['Close'].rolling(window=4).apply(lambda x: x.min() if x.idxmin() == x.index[-1] else x.max())

    return data

# Calculate technical indicators for the stock data
stock_data = calculate_technical_indicators(stock_data)

# Display the first few rows of the data with technical indicators
stock_data.head()

Price,Adj Close,Close,High,Low,Open,Volume,EMA_8,EMA_21,SMA_10,RSI_14,MACD,MACD_Signal,SAR
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,,
Date,,,,,,,,,,,,,
2022-01-03 00:00:00+00:00,179.076599,182.009995,182.880005,177.710007,177.830002,104487900,182.009995,182.009995,NaN,NaN,0.000000,0.000000,NaN
2022-01-04 00:00:00+00:00,176.803787,179.699997,182.940002,179.119995,182.630005,99310400,181.496662,181.799995,NaN,NaN,-0.184273,-0.036855,NaN
2022-01-05 00:00:00+00:00,172.100876,174.919998,180.169998,174.639999,179.610001,94537600,180.035181,181.174540,NaN,NaN,-0.707857,-0.171055,NaN
2022-01-06 00:00:00+00:00,169.227936,172.000000,175.300003,171.639999,172.699997,96904000,178.249585,180.340491,NaN,NaN,-1.342941,-0.405432,172.000000
2022-01-07 00:00:00+00:00,169.395187,172.169998,174.139999,171.029999,172.889999,86709100,176.898566,179.597719,NaN,NaN,-1.811647,-0.686675,179.699997


# Create Custom Plots
Use Plotly to create interactive candlestick charts with overlaid technical indicators.

In [5]:
# Create Custom Plots

def plot_with_indicators(data):
    # Create a plotly figure
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(x=data.index,
                                 open=data['Open'],
                                 high=data['High'],
                                 low=data['Low'],
                                 close=data['Close'],
                                 name='Candlestick'))

    # Add EMA lines
    fig.add_trace(go.Scatter(x=data.index, y=data['EMA_8'], mode='lines', name='EMA 8'))
    fig.add_trace(go.Scatter(x=data.index, y=data['EMA_21'], mode='lines', name='EMA 21'))

    # Add SMA line
    fig.add_trace(go.Scatter(x=data.index, y=data['SMA_10'], mode='lines', name='SMA 10'))

    # Add RSI line
    fig.add_trace(go.Scatter(x=data.index, y=data['RSI_14'], mode='lines', name='RSI 14', yaxis='y2'))

    # Add MACD lines
    fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], mode='lines', name='MACD'))
    fig.add_trace(go.Scatter(x=data.index, y=data['MACD_Signal'], mode='lines', name='MACD Signal'))

    # Add Parabolic SAR
    fig.add_trace(go.Scatter(x=data.index, y=data['SAR'], mode='markers', name='Parabolic SAR'))

    # Update layout
    fig.update_layout(title='Stock Price Data with Technical Indicators',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False,
                      yaxis2=dict(title='RSI', overlaying='y', side='right', position=0.15))

    fig.show()

# Plot the stock data with technical indicators
plot_with_indicators(stock_data)

In [7]:
def plot_with_indicators(data):
    # Create a figure with secondary y-axis
    fig = go.Figure()

    # Add candlestick chart with increased opacity and visible bars
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close'],
        name='OHLC',
        increasing_line_color='green',
        decreasing_line_color='red',
        showlegend=True,
        opacity=1.0
    ))

    # Add EMA lines
    fig.add_trace(go.Scatter(
        x=data.index, 
        y=data['EMA_8'], 
        mode='lines',
        name='EMA 8',
        line=dict(color='orange', width=1)
    ))
    
    fig.add_trace(go.Scatter(
        x=data.index, 
        y=data['EMA_21'], 
        mode='lines',
        name='EMA 21',
        line=dict(color='blue', width=1)
    ))

    # Add SMA line
    fig.add_trace(go.Scatter(
        x=data.index, 
        y=data['SMA_10'], 
        mode='lines',
        name='SMA 10',
        line=dict(color='purple', width=1)
    ))

    # Add RSI
    fig.add_trace(go.Scatter(
        x=data.index, 
        y=data['RSI_14'], 
        mode='lines',
        name='RSI 14',
        line=dict(color='gray', width=1),
        yaxis='y2'
    ))

    # Add MACD
    fig.add_trace(go.Scatter(
        x=data.index, 
        y=data['MACD'], 
        mode='lines',
        name='MACD',
        line=dict(color='blue', width=1),
        yaxis='y2'
    ))

    fig.add_trace(go.Scatter(
        x=data.index, 
        y=data['MACD_Signal'], 
        mode='lines',
        name='MACD Signal',
        line=dict(color='orange', width=1),
        yaxis='y2'
    ))

    # Add Parabolic SAR
    fig.add_trace(go.Scatter(
        x=data.index, 
        y=data['SAR'], 
        mode='markers',
        name='SAR',
        marker=dict(
            color='black',
            size=4,
            symbol='diamond'
        )
    ))

    # Update layout
    fig.update_layout(
        title='Stock Price Data with Technical Indicators',
        yaxis_title='Price',
        yaxis2=dict(
            title='Indicators',
            overlaying='y',
            side='right',
            position=0.95
        ),
        xaxis_title='Date',
        height=800,
        xaxis_rangeslider_visible=False,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Update y-axes ranges
    fig.update_yaxes(title_text="Price", secondary_y=False)
    fig.update_yaxes(title_text="Indicators", secondary_y=True)

    fig.show()

In [ ]:
plot

# Implement Interactive Display
Create a method to display the full analysis with customizable parameters like ticker symbol and time periods.

In [6]:
import ipywidgets as widgets
from IPython.display import display

def interactive_display():
    # Create widgets for user input
    ticker_widget = widgets.Text(value='AAPL', description='Ticker:')
    start_date_widget = widgets.DatePicker(value=pd.to_datetime('2022-01-01'), description='Start Date:')
    end_date_widget = widgets.DatePicker(value=pd.to_datetime('2023-01-01'), description='End Date:')
    button = widgets.Button(description='Plot')

    # Define the button click event
    def on_button_click(b):
        ticker = ticker_widget.value
        start_date = start_date_widget.value.strftime('%Y-%m-%d')
        end_date = end_date_widget.value.strftime('%Y-%m-%d')

        # Fetch the stock data
        stock_data = stock_plotter.get_data(ticker, start_date, end_date)

        # Calculate technical indicators
        stock_data = calculate_technical_indicators(stock_data)

        # Plot the stock data with technical indicators
        plot_with_indicators(stock_data)

    button.on_click(on_button_click)

    # Display the widgets
    display(ticker_widget, start_date_widget, end_date_widget, button)

# Call the interactive display function
interactive_display()

Text(value='AAPL', description='Ticker:')

DatePicker(value=Timestamp('2022-01-01 00:00:00'), description='Start Date:', step=1)

DatePicker(value=Timestamp('2023-01-01 00:00:00'), description='End Date:', step=1)

Button(description='Plot', style=ButtonStyle())

[*********************100%***********************]  1 of 1 completed
